# Mini-projet - Elections
*Basé sur Hands-on Data Preprocessing, R. Jafari, 2022*

**Ce TP est noté. Merci de lire attentivement le fichier instructions.pdf avant de commencer**

Nom étudiant 1: **Gombas**

Prénom étudiant 1: **Owen**

Nom étudiant 2: **Darmanger**

Prénom étudiant 2: **David**

## Préambule

### Imports

In [ ]:
# packages standards
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
from typing import List, Dict, Tuple, Callable, Any
import re

In [ ]:
# packages spécifiques
import matplotlib.image as mpimg
import seaborn as sns

### Data

In [ ]:
DATA_FOLDER = os.path.join(".", "data")  # chemin relatif et nom du dossier "data"

RAW_FOLDER = os.path.join(
    DATA_FOLDER, "raw"
)  # chemin du dossier raw (ne devrait pas être changé): INPUT

PREPROCESSED_FOLDER = os.path.join(
    DATA_FOLDER, "preprocessed"
)  # chemin du dossier preprocessed (resultat du traitement raw): OUTPUT

MEDIA_FOLDER = os.path.join(
    DATA_FOLDER, "media"
)  # chemin du dossier media pour les illustrations de mise en page des notebooks

EXPLORATION_FOLDER = os.path.join(
    DATA_FOLDER, "exploration"
)  # chemin du dossier exploration pour les notebooks d'exploration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import glob
from pandas_profiling import ProfileReport

In [ ]:
# dictionnary with filename and header row
files = {
    "Education.xls": dict(header=4, dtype={"FIPS Code": str}),
    "Unemployment.xlsx": dict(header=4, dtype={"FIPS_Code": str}),
    "PopulationEstimates.xls": dict(header=2, dtype={"FIPStxt": str}),
    "PovertyEstimates.xls": dict(header=4, dtype={"FIPStxt": str}),
    "countypres_2000-2020.csv": dict(header=0, dtype={"county_fips": str}),
}

files

# Les 3 niveaux de nettoyage
1. Les manipulation génériques (i.e., non spécifique à une application)
2. Les restructurations pour un outil d’analyse spécifique (ex.: Tableau, Excel, Plotly, SPSS, ...)
3. L’interprétation des valeurs et les restructurations pour un objectif d’analyse spécifique ou une problématique donnée, e.g.,
   1. «Quelle est le prix pour une maison de 200m2 dans ce secteur?»,
   2. «Quel accessoire recommander si l’utilisateur vient d’acheter ce téléphone portable?»
   3. «Quel est le lien entre les facteurs sociaux et le tabagisme?»
   
*Source: cours*

# Import des raw datasets et affichage

In [ ]:
dfs = {}

for file in files:
    basename, ext = file.split(".")

    if ext == "xls" or ext == "xlsx":
        df = pd.read_excel(os.path.join(RAW_FOLDER, file), header=files[file]["header"], dtype=files[file]["dtype"])
        dfs[basename] = df
    elif ext == "csv":
        df = pd.read_csv(os.path.join(RAW_FOLDER, file), header=files[file]["header"], dtype=files[file]["dtype"])
        dfs[basename] = df

dfs

In [ ]:
dfs["countypres_2000-2020"]

In [ ]:
dfs["Education"]

In [ ]:
dfs["PovertyEstimates"]

In [ ]:
dfs["Unemployment"]

In [ ]:
dfs["PopulationEstimates"]

# Descriptions de chacun des datasets à des fins de compréhension

In [ ]:
def describe_df(df):
    print("Shape:", df.shape, end="\n\n")
    print(f"{len(df.columns)} columns:", df.columns, end="\n\n")
    print("Index:", df.index, end="\n\n")
    print("Dtypes:", df.dtypes, end="\n\n")
    print("Nulls:", df.isnull().sum(), end="\n\n")
    print("Unique values:", df.nunique(), end="\n\n")

In [ ]:
for key in dfs:
    print(key)
    describe_df(dfs[key])
    print("\n" * 4 + "=" * 100 + "\n" * 4)

# Compte combien de valeurs chaque colonnes possèdes

In [ ]:
for key in dfs:
    fig = plt.figure(figsize=(30, 10))
    plt.title(key)
    dfs[key].count().plot(kind="bar")

# Show NaN

In [ ]:
for key in dfs:
    fig = plt.figure(figsize=(30, 10))
    plt.title(key)
    dfs[key].isnull().sum().plot(kind="bar")

# Affichage de combien de valeurs uniques chaque colonnes possède

In [ ]:
for key in dfs:
    fig = plt.figure(figsize=(30, 10))
    plt.title(key)
    dfs[key].nunique().plot(kind="bar")

# Valeurs manquantes
Pour choisir, il faut comprendre l’objectif de l’analyse et l’impact des valeurs manquantes

- (Les laisser manquantes)
- Supprimer les lignes concernées
- Supprimer les colonnes concernées
- Estimer et imputer une valeur

## Types de valeurs manquantes

### Manquante aléatoirement
- Pas de cause systématique (aléa(s) unique(s) ou dysfonctionnement aléatoire)
- Elles sont inévitables

### Manquante systématiquement
- Cause systématique, potentiellement prédictible
- On peut les éviter en changeant le processus de collecte

*Source: cours*
  
## Traitement des valeurs manquantes et normalisation des dataframes

In [ ]:
def is_state(df: pd.DataFrame, fips_column: str = "fips"):
    return df[fips_column].apply(lambda x: str(x)[-3:] == "000")

In [ ]:
def delete_us(df: pd.DataFrame):
    return df[df["state"] != "US"]

In [ ]:
def delete_states(df: pd.DataFrame, fips_column: str = "fips"):
    return df[~is_state(df, fips_column)]

In [ ]:
def delete_pr(df: pd.DataFrame):
    return df[df["state"] != "PR"]

In [ ]:
def fill_numeric_columns(df: pd.DataFrame, method=Callable[[pd.DataFrame], Any]):
    numeric_cols = df.select_dtypes(include=np.number).columns
    df.loc[:, numeric_cols] = df[numeric_cols].fillna(method(df[numeric_cols]))
    return df

In [ ]:
def save_df(df: pd.DataFrame, name: str):
    df.to_csv(os.path.join(PREPROCESSED_FOLDER, name + ".csv"), index=False)
    return df

In [ ]:
def show_na(df: pd.DataFrame):
    nulls = df.isnull().sum()[df.isnull().sum() > 0].to_dict()
    
    if len(nulls) == 0:
        print("No null values")
        return 0

    for key, value in nulls.items():
        print(f"{key}: {value}")
    
    # show rows with null values
    nulls_df = df[df.isnull().any(axis=1)]

    # print rows of nulls_df
    for i in range(len(nulls_df)):
        r = ""
        for key, value in nulls_df.iloc[i].to_dict().items():
            r += f"{key}: {value} | "
        print(r)

    fig = plt.figure(figsize=(30, 10))
    df.isnull().sum().plot(kind="bar")

    return len(nulls_df)

In [ ]:
def normalize_columns(df: pd.DataFrame, fips_column: str = "fips", state_column: str = "state"):
    if state_column in df.columns:
        df.rename(columns={state_column: "state"}, inplace=True)

    if fips_column in df.columns:
        df[fips_column] = df[fips_column].apply(lambda x: str(x).zfill(5) if x else None).astype(str)
        df.rename(columns={fips_column: "fips"}, inplace=True)

    df.columns = [col.lower().replace(" ", "_") for col in df.columns]
    df.columns = [col.replace("-", "_") for col in df.columns]
    df.columns = [re.sub(r"['(),]", "", col) for col in df.columns]

    return df

In [ ]:
def replace_hard_on_fips(df: pd.DataFrame, fips: str, column: str, value: Any):
    df.loc[df["fips"] == fips, column] = value
    return df

### Education
- Manquantes car l'état en lui même ne contient pas toutes les informations, ce sont les villes dans les états qui les possèdes -> Supprimer
- Ne possède pas l'information -> Imputer avec la moyenne de la colonne

In [ ]:
education_df: pd.DataFrame = dfs["Education"]
education_df = normalize_columns(education_df, "FIPS Code")
education_df = delete_us(education_df)
education_df = delete_states(education_df)
education_df = delete_pr(education_df)
education_df = fill_numeric_columns(education_df, method=np.mean)

In [ ]:
assert show_na(education_df) == 0

In [ ]:
save_df(education_df, "education")

### PopulationEstimates
- Manquantes car l'état en lui même ne contient pas toutes les informations, ce sont les villes dans les états qui les possèdes -> Supprimer
- Ne possède pas l'information -> Dépend de la nature de la colonne

In [ ]:
population_estimates_df: pd.DataFrame = dfs["PopulationEstimates"]
population_estimates_df = normalize_columns(population_estimates_df, "FIPStxt")
population_estimates_df = delete_us(population_estimates_df)
population_estimates_df = delete_states(population_estimates_df)
population_estimates_df

In [ ]:
show_na(population_estimates_df)

**Rural-Urban Continuum Codes (https://www.ers.usda.gov/data-products/rural-urban-continuum-codes.aspx)**  
The 2013 Rural-Urban Continuum Codes form a classification scheme that distinguishes metropolitan counties by the population size of their metro area, and nonmetropolitan counties by degree of urbanization and adjacency to a metro area. The official Office of Management and Budget (OMB) metro and nonmetro categories have been subdivided into three metro and six nonmetro categories. Each county in the U.S., municipio in Puerto Rico, and Census Bureau-designated county-equivalent area of the Virgin Islands/other inhabited island territories of the U.S. is assigned one of the 9 codes. This scheme allows researchers to break county data into finer residential groups, beyond metro and nonmetro, particularly for the analysis of trends in nonmetro areas that are related to population density and metro influence. The Rural-Urban Continuum Codes were originally developed in 1974. They have been updated each decennial since (1983, 1993, 2003, 2013), and slightly revised in 1988. Note that the 2013 Rural-Urban Continuum Codes are not directly comparable with the codes prior to 2000 because of the new methodology used in developing the 2000 metropolitan areas.

**Urban Influence Codes (https://www.ers.usda.gov/data-products/urban-influence-codes/)**  
The 2013 Urban Influence Codes form a classification scheme that distinguishes metropolitan counties by population size of their metro area, and nonmetropolitan counties by size of the largest city or town and proximity to metro and micropolitan areas. The standard Office of Management and Budget (OMB) metro and nonmetro categories have been subdivided into two metro and 10 nonmetro categories, resulting in a 12-part county classification. This scheme was originally developed in 1993. This scheme allows researchers to break county data into finer residential groups, beyond metro and nonmetro, particularly for the analysis of trends in nonmetro areas that are related to population density and metro influence. 

**Rural-urban_Continuum Code_2003 (source: https://seer.cancer.gov/seerstat/variables/countyattribs/Rural.Urban.Continuum.Codes.1974.1983.1993.2003.2013.pdf)**
- FIPStxt: 2105 | State: AK -> 9
- FIPStxt: 2195 | State: AK -> 9
- FIPStxt: 2198 | State: AK -> 9
- FIPStxt: 2230 | State: AK -> 9
- FIPStxt: 2275 | State: AK -> 9

**Urban_Influence_Code_2003 (source: https://www.ers.usda.gov/data-products/urban-influence-codes/)**
- FIPStxt: 2105 | State: AK -> 10
- FIPStxt: 2195 | State: AK -> 11
- FIPStxt: 2198 | State: AK -> 10
- FIPStxt: 2230 | State: AK -> 12
- FIPStxt: 2275 | State: AK -> 12

In [ ]:
population_estimates_df = replace_hard_on_fips(population_estimates_df, "02105", "rural_urban_continuum_code_2003", 9)
population_estimates_df = replace_hard_on_fips(population_estimates_df, "02195", "rural_urban_continuum_code_2003", 9)
population_estimates_df = replace_hard_on_fips(population_estimates_df, "02198", "rural_urban_continuum_code_2003", 9)
population_estimates_df = replace_hard_on_fips(population_estimates_df, "02230", "rural_urban_continuum_code_2003", 9)
population_estimates_df = replace_hard_on_fips(population_estimates_df, "02275", "rural_urban_continuum_code_2003", 9)

population_estimates_df = replace_hard_on_fips(population_estimates_df, "02105", "urban_influence_code_2003", 10)
population_estimates_df = replace_hard_on_fips(population_estimates_df, "02195", "urban_influence_code_2003", 11)
population_estimates_df = replace_hard_on_fips(population_estimates_df, "02198", "urban_influence_code_2003", 10)
population_estimates_df = replace_hard_on_fips(population_estimates_df, "02230", "urban_influence_code_2003", 12)
population_estimates_df = replace_hard_on_fips(population_estimates_df, "02275", "urban_influence_code_2003", 12)

**Puerto Rico (PR) manque trop de valeurs, nous ne pouvons pas les imputer avec des valeurs pertinente, nous abandonnons alors le Puerto Rico**

In [ ]:
population_estimates_df = delete_pr(population_estimates_df)

In [ ]:
assert show_na(population_estimates_df) == 0

In [ ]:
save_df(population_estimates_df, "PopulationEstimates")

### PovertyEstimates

In [ ]:
poverity_estimates_df: pd.DataFrame = dfs["PovertyEstimates"]
poverity_estimates_df = normalize_columns(poverity_estimates_df, "FIPStxt", "Stabr")
poverity_estimates_df = delete_us(poverity_estimates_df)
poverity_estimates_df = delete_states(poverity_estimates_df)
poverity_estimates_df = delete_pr(poverity_estimates_df)
poverity_estimates_df

In [ ]:
show_na(poverity_estimates_df)

**rural_urban_continuum_code_2003 basé sur celui de 2013 (source: https://www.ers.usda.gov/data-products/rural-urban-continuum-codes.aspx)**
- fips: 02105 | state: AK -> 10
- fips: 02195 | state: AK -> 11
- fips: 02198 | state: AK -> 10
- fips: 02230 | state: AK -> 12
- fips: 02275 | state: AK -> 12

In [ ]:
poverity_estimates_df = replace_hard_on_fips(poverity_estimates_df, "02105", "rural_urban_continuum_code_2003", 10)
poverity_estimates_df = replace_hard_on_fips(poverity_estimates_df, "02195", "rural_urban_continuum_code_2003", 11)
poverity_estimates_df = replace_hard_on_fips(poverity_estimates_df, "02198", "rural_urban_continuum_code_2003", 10)
poverity_estimates_df = replace_hard_on_fips(poverity_estimates_df, "02230", "rural_urban_continuum_code_2003", 12)
poverity_estimates_df = replace_hard_on_fips(poverity_estimates_df, "02275", "rural_urban_continuum_code_2003", 12)

**urban_influence_code_2003 basé sur celui de 2013 (source: https://www.ers.usda.gov/data-products/rural-urban-continuum-codes.aspx)**
- fips: 02105 | state: AK -> 10
- fips: 02195 | state: AK -> 11
- fips: 02198 | state: AK -> 10
- fips: 02230 | state: AK -> 12
- fips: 02275 | state: AK -> 12

In [ ]:
poverity_estimates_df = replace_hard_on_fips(poverity_estimates_df, "02105", "urban_influence_code_2003", 10)
poverity_estimates_df = replace_hard_on_fips(poverity_estimates_df, "02195", "urban_influence_code_2003", 11)
poverity_estimates_df = replace_hard_on_fips(poverity_estimates_df, "02198", "urban_influence_code_2003", 10)
poverity_estimates_df = replace_hard_on_fips(poverity_estimates_df, "02230", "urban_influence_code_2003", 12)
poverity_estimates_df = replace_hard_on_fips(poverity_estimates_df, "02275", "urban_influence_code_2003", 12)

**pov04_2019, ci90lb04_2019, ci90ub04_2019, pctpov04_2019, ci90lb04p_2019, ci90ub04p_2019**  
Colonnes complètements vides -> suppression

In [ ]:
poverity_estimates_df = poverity_estimates_df.drop(columns=[
    "pov04_2019",
    "ci90lb04_2019",
    "ci90ub04_2019",
    "pctpov04_2019",
    "ci90lb04p_2019",
    "ci90ub04p_2019"
])

In [ ]:
assert show_na(poverity_estimates_df) == 0

In [ ]:
save_df(poverity_estimates_df, "PovertyEstimates")

## Unemployment

In [ ]:
unemployment_df: pd.DataFrame = dfs["Unemployment"]
unemployment_df = normalize_columns(unemployment_df, "FIPS_Code")
unemployment_df = delete_us(unemployment_df)
unemployment_df = delete_states(unemployment_df)
unemployment_df = delete_pr(unemployment_df)
unemployment_df

In [ ]:
show_na(unemployment_df)

**Pour toutes ces valeurs on peut les imputer avec la moyenne de leurs états respectifs**

In [ ]:
numeric_cols = unemployment_df.select_dtypes(include=np.number).columns
unemployment_df[numeric_cols] = unemployment_df.groupby("state")[numeric_cols].transform(lambda x: x.fillna(x.mean()))

unemployment_df

In [ ]:
assert show_na(unemployment_df) == 0

In [ ]:
save_df(unemployment_df, "Unemployment")

## countypres

In [ ]:
countypres_df: pd.DataFrame = dfs["countypres_2000-2020"]
countypres_df.rename(columns={"state": "full_state_name"}, inplace=True)
countypres_df = normalize_columns(countypres_df, "county_fips", "state_po")
countypres_df

In [ ]:
show_na(countypres_df)

In [ ]:
missing_states = {
    "connecticut": "CT",
    "alaska": "AK",
    "rhode island": "RI",
    "maine": "ME",
}

countypres_df["state"] = countypres_df["full_state_name"].apply(lambda x: missing_states[x] if x in missing_states else x)

show_na(countypres_df)

In [ ]:
save_df(countypres_df, "countypres_2000-2020")

### Un fips unique